## The process of getting the model of this article

In [1]:
#Importing related packages
from numpy import loadtxt
from xgboost import XGBClassifier
from matplotlib import pyplot
from xgboost import plot_importance
import pandas as pd
import xgboost as xgb
from numpy import *
from sklearn.metrics import mean_absolute_error,r2_score
import numpy as np
import catboost
from catboost import CatBoostRegressor  
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from deap import base, creator, tools, algorithms
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb

In [2]:
def devide_dataset(x,y):                    # Delineating the data set
    tr_x1,te_x1,tr_y1,te_y1=train_test_split(x,y,test_size=0.1,random_state=5)
    return tr_x1,te_x1,tr_y1,te_y1

def mape(actual, pred):                     # Computing mape
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) 

def mae(actual,pred):                       # Computing mae
    actual,pred=np.array(actual),np.array(pred)
    a=np.mean(np.abs(actual-pred))
    return a

def rmse(actual,pred):                      # Computing rmse
    actual,pred=np.array(actual),np.array(pred)
    a=actual-pred
    b=np.mean(np.power(a, 2))
    c=np.sqrt(b)
    return c

def mse(actual,pred):                       # Computing mse
    actual,pred=np.array(actual),np.array(pred)
    a=actual-pred
    b=np.mean(np.power(a, 2))
    return b

In [3]:
df1=pd.read_excel("D:\\pyplt\\cat_process.xlsx")# Importing the original data set  
# Separate variable and target factors
X=df1[['Year', 'Geographic Region', 'Length \n(ft)', 'LWL (ft)',
       'Beam (ft)', 'Draft (ft)', 'Displacement (lbs)', 'Sail Area (sq ft)',
       'Average Cargo Throughput (tons)', 'GDP ($bn)', 'GDP per capita ($)',
       'Average ratio of total logistics cost to GDP',  'DLR',
      'MSR', 'CC', 'LBR', 'MCR']]
Y=df1[['Listing Price (USD)']]

In [4]:
#Generate a new dataset based on the results of feature filtering
cat_x=df1[['Year', 'Length \n(ft)', 'LWL (ft)',
       'Displacement (lbs)', 'Sail Area (sq ft)',
       'GDP ($bn)', 
       'DLR',
      'MSR', 'CC','MCR']]
xg_x=df1[['Year',  'Length \n(ft)', 'LWL (ft)',
       'Draft (ft)', 'Displacement (lbs)', 'Sail Area (sq ft)',
       'GDP ($bn)',
       'DLR',
       'LBR', 'MCR']]
y=Y['Listing Price (USD)']
tr_x,te_x,tr_y,te_y=devide_dataset(cat_x,y)
tr_x1,te_x1,tr_y1,te_y1=devide_dataset(xg_x,y)

In [44]:
import numpy as np
import random 
from sklearn.metrics import f1_score
import xgboost 

class GeneticXgboost:
    def __init__(self,num_parents=None):
        """
        param num_parents:The number of individuals in the population
        
        """
        self.num_parents = num_parents
    
    
    def initilialize_poplulation(self):
        """
        Initialize a population, that is, generate genes for a specified number of populations  
        
        The optimization parameters are as follows：
        
        1、learning_rate:As an important hyperparameter in supervised learning and deep learning,
                         it determines whether and when the objective function can converge to the local minimum
        
        2、n_estimators:The number of subdatasets generated by putback sampling of the original dataset
        
        3、max_depth:Maximum depth of a tree, increase this value will make model more complex 
        
        4、min_child_weight:In linear regression mode, this simply corresponds to minimum number of instances 
                            needed to be in each node.
        5、subsample:Subsample ratio of the training instance.
        
        6、olsample_bytree:Subsample ratio of columns when constructing each tree.
        
        7、gamma:minimum Loss reduction required to make a further partition on a leaf node of the tree. 
        
        return：array,shape=[self.num_parents,num_gene]        
        """
        learningRate = np.empty([self.num_parents, 1])
        maxDepth = np.empty([self.num_parents, 1],dtype = np.uint8)
        minChildWeight = np.empty([self.num_parents,1])
        gammaValue = np.empty([self.num_parents,1])
        subSample = np.empty([self.num_parents,1])
        colSampleByTree = np.empty([self.num_parents,1])
        for i in range(self.num_parents): 
            #Generate each individual
            learningRate[i]    = round(np.random.uniform(0.001, 1), 3)
            maxDepth[i]        = int(random.randrange(1, 12, step=1))
            minChildWeight[i]  = round(random.uniform(0.001, 15.00),3)
            gammaValue[i]      = round(random.uniform(0.001, 15.00),3)
            subSample[i]       = round(random.uniform(0.001, 1.00), 3)
            colSampleByTree[i] = round(random.uniform(0.001, 1.00), 3)
            population = np.concatenate((learningRate,maxDepth,minChildWeight,
                                         gammaValue,subSample,colSampleByTree),axis=1)
        return population
    
    def fitness_function(self,y_true,y_pred):
        """
        Define the fitness function
        This paper uses MABE as a fitness function
        """
        fitness = round((mape(y_true,y_pred)),6)
        return fitness
    
    
    def fitness_compute(self,population,dMatrixTrain,dMatrixtest,y_test):
        """
        Calculate the fitness value
        param population:  Population
        param dMatrixTrain:Training data,(X,y)
        param dMatrixtest: test data,(x,y)
        param y_test:      Test data,y
        return The fitness value of each individual in the population               
        """
        mse_Score = []
        for i in range(population.shape[0]):#Traverse every individual in the population
            param = {'objective':       'reg:squarederror',
                     'n_estimators':    600,
                     'learning_rate':    population[i][0],
                     'max_depth':        int(population[i][1]), 
                     'min_child_weight': population[i][2],
                     'gamma':            population[i][3], 
                     'subsample':        population[i][4],
                     'colsample_bytree': population[i][5],
                     'seed': 24}
            num_round = 100
            model = xgboost.train(param,dMatrixTrain,num_round)
            preds = model.predict(dMatrixtest)
            
            mse = self.fitness_function(y_test,preds)
            mse_Score.append(mse)
        return mse_Score
    
    def parents_selection(self,population,fitness,num_store):
        """
        The number of individuals in the retained population is selected based on the fitness value
        
        param population:Population，shape=[self.num_parents,num_gene]
        param num_store: The number of individuals that need to be retained  
        param fitness:   Fitness value，array
        
        return The best preserved individual in the population，shape=[num_store,num_gene]
        """
        #Used to store individuals that need to be retained
        selectedParents = np.empty((num_store,population.shape[1])) 
        for parentId in range(num_store):
            #Find the index of the smallest value
            bestFitnessId = np.where(fitness == np.min(fitness))
            bestFitnessId = bestFitnessId[0][0]
            #Save the corresponding individual genes
            selectedParents[parentId,:] = population[bestFitnessId, :]
            #Assign +2 to the minimum fitness of the extracted value to avoid extracting it again
            fitness[bestFitnessId] = +2
            
        return selectedParents
    
    def crossover_uniform(self,parents,childrenSize):
        """
        cross
             We use uniform crossing,in which each parameter of the child 
             will be independently selected from the parents based on a specific distribution
             
        param parents
        param childrenSize
        
        return         
        """
        
        crossoverPointIndex = np.arange(0,np.uint8(childrenSize[1]),1,dtype= np.uint8)
        
        crossoverPointIndex1 = np.random.randint(0,np.uint8(childrenSize[1]),
                                                 np.uint8(childrenSize[1]/2))
        
        crossoverPointIndex2 = np.array(list(set(crossoverPointIndex)-set(crossoverPointIndex1)))
        
        children = np.empty(childrenSize)
        
        #Cross the two parent individuals
        for i in range(childrenSize[0]): 
            #find parent1 index 
            parent1_index = i%parents.shape[0]
            #find parent 2 index
            parent2_index = (i+1)%parents.shape[0]
            #insert parameters based on random selected indexes in parent1
            children[i,crossoverPointIndex1] = parents[parent1_index,crossoverPointIndex1]
            #insert parameters based on random selected indexes in parent1
            children[i,crossoverPointIndex2] = parents[parent2_index,crossoverPointIndex2]
        return children
    
    def mutation(self, crossover, num_param):
        '''
        mutation
          Randomly select a parameter and introduce diversity in offspring 
          by changing the value by a random amount
        param crossover:The mutated population
        param num_param:The number of parameters
        return         
        '''
        
        #Define the minimum and maximum values allowed for each parameter
        minMaxValue = np.zeros((num_param,2))
        
        minMaxValue[0,:] = [0.01, 1.0]  #min/max learning rate
        minMaxValue[1,:] = [1, 15]      #min/max depth
        minMaxValue[2,:] = [0, 15.00]    #min/max child_weight
        minMaxValue[3,:] = [0.001, 15.00] #min/max gamma
        minMaxValue[4,:] = [0.001, 1.00]  #min/maxsubsample
        minMaxValue[5,:] = [0.001, 1.00]  #min/maxcolsample_bytree
        
        #The mutation randomly changes a single gene in each offspring
        mutationValue = 0
        parameterSelect = np.random.randint(0,6,1)
        
        if parameterSelect == 0: 
            #learning_rate
            mutationValue = round(np.random.uniform(-0.01, 0.01), 3)
        if parameterSelect == 1: 
            #max_depth
            mutationValue = np.random.randint(-2, 2, 1)
        if parameterSelect == 2: 
            #min_child_weight
            mutationValue = round(np.random.uniform(2.00, 3.00), 3)
        if parameterSelect == 3: 
            #gamma
            mutationValue = round(np.random.uniform(-1, 1), 3)
        if parameterSelect == 4: 
            #subsample
            mutationValue = round(np.random.uniform(-0.2, 0.2), 3)
        if parameterSelect == 5: 
            #colsample
            mutationValue = round(np.random.uniform(-0.2, 0.2), 3)
            
        #Introduce variation by changing one parameter and set to max or min if out of range
        for idx in range(crossover.shape[0]):
            crossover[idx, parameterSelect] = crossover[idx,parameterSelect]+mutationValue
            
            if(crossover[idx,parameterSelect]>minMaxValue[parameterSelect,1]):            
                crossover[idx,parameterSelect] = minMaxValue[parameterSelect,1]
            
            if(crossover[idx,parameterSelect] < minMaxValue[parameterSelect,0]):
                crossover[idx,parameterSelect] = minMaxValue[parameterSelect,0]
            
        return crossover    


######################Parametric shrinkage test##############################################

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#tr_x1,tr_y1
#te_x1,te_y1

xgDMatrixTrain = xgboost.DMatrix(tr_x1,tr_y1)
xgbDMatrixTest = xgboost.DMatrix(te_x1,te_y1)


number_of_parents = 150        #Initial population size
number_of_generations = 5      #Population reproduction algebra, that is, the number of iterations
number_of_parameters = 6       #The number of parameters that will be optimized
number_of_parents_mating = 4   #The number of individuals retained per generation

gx = GeneticXgboost(num_parents=number_of_parents)

#Define the size of the population
populationSize = (number_of_parents,number_of_parameters)

#Initial population
population = gx.initilialize_poplulation()
#Define an array to store fitness history
FitnessHistory = np.empty([number_of_generations+1, number_of_parents])
#Define an array to store the values of each parent node and each generated parameter
populationHistory = np.empty([(number_of_generations+1)*number_of_parents,
                               number_of_parameters])
#The value of the initial parameter inserted in the history
populationHistory[0:number_of_parents,:] = population

#train
for generation in range(number_of_generations):
    print("This is number %s generation" %(generation))
    #train the dataset and obtain fitness
    FitnessValue = gx.fitness_compute(population=population,
                                      dMatrixTrain=xgDMatrixTrain, 
                                      dMatrixtest=xgbDMatrixTest, 
                                      y_test=te_y1)
    
#     FitnessHistory[generation,:] = FitnessValue
#     print('Best F1 score in the iteration = {}'.format(np.max(FitnessHistory[generation,:])))
    #Reserved parent
    parents = gx.parents_selection(population=population,
                                                   fitness=FitnessValue,
                                                   num_store=number_of_parents_mating)
    #Generated offspring
    children = gx.crossover_uniform(parents=parents, 
                     childrenSize=(populationSize[0]-parents.shape[0],number_of_parameters))
    
    #Increase mutations to create genetic diversity
    children_mutated = gx.mutation(children, number_of_parameters)
    
    '''
       Create a new population that will contain the parent previously selected based on the fitness value, 
       and the resulting offspring

    '''
    population[0:parents.shape[0], :] = parents 
    population[parents.shape[0]:,  :] = children_mutated
    populationHistory[(generation+1)*number_of_parents:(generation+1)*number_of_parents+number_of_parents,:]=population
    

#The best solution for the final iteration  
fitness = gx.fitness_compute(population=population, 
                             dMatrixTrain=xgDMatrixTrain, 
                             dMatrixtest=xgbDMatrixTest, 
                             y_test=te_y1)

bestFitnessIndex = np.where(fitness == np.min(fitness))[0][0]
print("Best fitness is =", fitness[bestFitnessIndex])

print("Best parameters are:")
print('n_estimators',          600)
print('learning_rate=',        population[bestFitnessIndex][0])
print('max_depth=',            int(population[bestFitnessIndex][1])) 
print('min_child_weight=',     population[bestFitnessIndex][2])
print('gamma=',                population[bestFitnessIndex][3])
print('subsample=',            population[bestFitnessIndex][4])
print('colsample_bytree=',     population[bestFitnessIndex][5]) 

This is number 0 generation
[20:46:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[20:46:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[20:46:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[20:46:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[20:46:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Paramete

In [16]:
import numpy as np
import random 
from sklearn.metrics import f1_score
import xgboost 
import catboost as cb
from catboost import Pool
class GeneticCatboost:
    def __init__(self,num_parents=None):
        """
        param num_parents:The number of individuals in the population
        
        """
        self.num_parents = num_parents
    
    
    def initilialize_poplulation(self):
        """
         Initialize a population, that is, generate genes for a specified number of populations  
        
         The optimization parameters are as follows： 
         1、depth :Depth of the tree.
         
         2、iterations：The maximum number of trees that can be built when solving machine learning problems.
         
         3、learning_rate:Used for reducing the gradient step.
         
         4、l2_leaf_reg:Coefficient at the L2 regularization term of the cost function.
         
         5、bagging_temperature:Defines the settings of the Bayesian bootstrap. 
                                It is used by default in classification and regression modes.
         6、random_strength:The amount of randomness to use for scoring splits when the tree structure is selected. 
                            Use this parameter to avoid overfitting the model.
        
         return：array,shape=[self.num_parents,num_gene]        
        """
        depth = np.empty([self.num_parents, 1])
        iterations  = np.empty([self.num_parents, 1],dtype = np.uint8)
        learning_rate = np.empty([self.num_parents, 1])
        l2_leaf_reg = np.empty([self.num_parents,1])
        bagging_temperature = np.empty([self.num_parents,1])
        random_strength = np.empty([self.num_parents,1])
      
        for i in range(self.num_parents): 
            #Generate each individual
            learning_rate [i]    = round(np.random.uniform(0.01, 1), 2)
            iterations[i]     = int(random.randrange(10, 1500, step = 25))
            depth[i]        = int(random.randrange(1, 10, step=1))
            l2_leaf_reg[i]  = round(random.uniform(0.01, 10.0),2)
            bagging_temperature[i]      = round(random.uniform(0.01, 10.0),2)
            random_strength[i]       = round(random.uniform(0.01, 1.0), 2)
            
            #Synthesize a complete population
            population = np.concatenate((depth,iterations, learning_rate,
                                         l2_leaf_reg,bagging_temperature,
                                       bagging_temperature,random_strength ,
                                     ),axis=1)
        return population
    
    def fitness_function(self,y_true,y_pred):
        """
        Define the fitness function
        This paper uses MABE as a fitness function
        """
        fitness = round((mape(y_true,y_pred)),6)
        return fitness
    
    def fitness_compute(self,population,dMatrixTrain,dMatrixtest,y_test):
        """
        Calculate the fitness value
        param population:  Population
        param dMatrixTrain:Training data,（X,y)
        param dMatrixtest: test data,(x,y)
        param y_test:      Test data,y
        return The fitness value of each individual in the population                
        """
        mape_Score = []
        """
        The columns represent the individual genes, 
        and the rows represent the population to which the individual belongs
         
        """
        
        for i in range(population.shape[0]):#Traverse every individual in the population
            params = {'silent':       True,
                     'depth':    int(population[i][0]),
                     'iterations':     population[i][1], 
                     'learning_rate':        population[i][2], 
                     'l2_leaf_reg': population[i][3],
                     'bagging_temperature':            population[i][4], 
                     'random_strength':        population[i][5],
                    
                     }
            model=catboost.CatBoostRegressor(**params)
            model.fit(dMatrixTrain)
            preds = model.predict(dMatrixtest)
            #Calculate the fitness functio
            mape = self.fitness_function(y_test,preds)
            mape_Score.append(mape)
        return mape_Score
    
    def parents_selection(self,population,fitness,num_store):
        """
        The number of individuals in the retained population is selected based on the fitness value
        param population:Population，shape=[self.num_parents,num_gene]
        param num_store: The number of individuals that need to be retained  
        param fitness:   Fitness value，array
        return The best preserved individual in the population，shape=[num_store,num_gene]
        """
        #Used to store individuals that need to be retained
        selectedParents = np.empty((num_store,population.shape[1])) 
        for parentId in range(num_store):
            #Find the index of the smallest value
            bestFitnessId = np.where(fitness == np.min(fitness))
            bestFitnessId = bestFitnessId[0][0]
            #Save the corresponding individual genes
            selectedParents[parentId,:] = population[bestFitnessId, :]
            #Assign +2 to the minimum fitness of the extracted value to avoid extracting it again
            fitness[bestFitnessId] = +1
            
        return selectedParents
    
    def crossover_uniform(self,parents,childrenSize):
        """
        cross
             We use uniform crossing,in which each parameter of the child 
             will be independently selected from the parents based on a specific distribution
        param parents:
        param childrenSize:
        return         
        """
        
        crossoverPointIndex = np.arange(0,np.uint8(childrenSize[1]),1,dtype= np.uint8)
        
        crossoverPointIndex1 = np.random.randint(0,np.uint8(childrenSize[1]),
                                                 np.uint8(childrenSize[1]/2))
        
        crossoverPointIndex2 = np.array(list(set(crossoverPointIndex)-set(crossoverPointIndex1)))
        
        children = np.empty(childrenSize)
        
         #Cross the two parent individuals
        for i in range(childrenSize[0]): 
            #find parent1 index 
            parent1_index = i%parents.shape[0]
            #find parent 2 index
            parent2_index = (i+1)%parents.shape[0]
            #insert parameters based on random selected indexes in parent1
            children[i,crossoverPointIndex1] = parents[parent1_index,crossoverPointIndex1]
            #insert parameters based on random selected indexes in parent1
            children[i,crossoverPointIndex2] = parents[parent2_index,crossoverPointIndex2]
        return children
    
    def mutation(self, crossover, num_param):
        '''
        mutation:
          Randomly select a parameter and introduce diversity in offspring 
          by changing the value by a random amount
        param crossover:The mutated population
        param num_param:The number of parameters
        return         
        '''
        
        #Define the minimum and maximum values allowed for each parameter
        minMaxValue = np.zeros((num_param,2))
        
        minMaxValue[0,:] = [1, 15] 
        minMaxValue[1,:] = [10, 2000]    
        minMaxValue[2,:] = [0.01, 1.0]      
        minMaxValue[3,:] = [0, 10.0]     
        minMaxValue[4,:] = [0.01, 10.0]  
        minMaxValue[5,:] = [0.01, 1.0]   
        
        
        #The mutation randomly changes a single gene in each offspring
        mutationValue = 0
        parameterSelect = np.random.randint(0,6,1)
    
        if parameterSelect == 0: 
            
            mutationValue = np.random.randint(-5, 5, 1)
        if parameterSelect == 1: 
             
            mutationValue = np.random.randint(-200, 200, 1)
        if parameterSelect == 2: 
             
            mutationValue = round(np.random.uniform(-0.5, 0.5), 2)
        if parameterSelect == 3: 
             
            mutationValue = round(np.random.uniform(5, 5), 2)
        if parameterSelect == 4: 
            
            mutationValue = round(np.random.uniform(-2, 2), 2)
        if parameterSelect == 5: 
             
            mutationValue = round(np.random.uniform(-0.5, 0.5), 2)
        
            
         #Introduce variation by changing one parameter and set to max or min if out of range
        for idx in range(crossover.shape[0]):
            crossover[idx, parameterSelect] = crossover[idx,parameterSelect]+mutationValue
            
            if(crossover[idx,parameterSelect]>minMaxValue[parameterSelect,1]):            
                crossover[idx,parameterSelect] = minMaxValue[parameterSelect,1]
            
            if(crossover[idx,parameterSelect] < minMaxValue[parameterSelect,0]):
                crossover[idx,parameterSelect] = minMaxValue[parameterSelect,0]
            
        return crossover    


######################Parametric shrinkage test##############################################

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#tr_x1,tr_y1
#te_x1,te_y1
np.array(tr_x)
dtrain = cb.Pool(tr_x, tr_y)
dtest  = cb.Pool(te_x, te_y)



number_of_parents = 50       #Initial population size
number_of_generations = 5     #Population reproduction algebra, that is the number of iterations
number_of_parameters = 7      #The number of parameters that will be optimized
number_of_parents_mating = 4  #The number of individuals retained per generation

gc = GeneticCatboost(num_parents=number_of_parents)

#Define the size of the population
populationSize = (number_of_parents,number_of_parameters)

#Initial population
population = gc.initilialize_poplulation()

#Define an array to store fitness history
FitnessHistory = np.empty([number_of_generations+1, number_of_parents])

#Define an array to store the values of each parent node and each generated parameter
populationHistory = np.empty([(number_of_generations+1)*number_of_parents,
                               number_of_parameters])

#The value of the initial parameter inserted in the history
populationHistory[0:number_of_parents,:] = population

#train
for generation in range(number_of_generations):
    print("This is number %s generation" %(generation))
    dMatrixTrain=dtrain
    dMatrixtest=dtest
    #train the dataset and obtain fitness
    FitnessValue = gc.fitness_compute(population,
                                      dMatrixTrain, 
                                      dMatrixtest, 
                                      te_y)
    
    FitnessHistory[generation,:] = FitnessValue

     #Reserved parent
    parents = gc.parents_selection(population=population,
                                                   fitness=FitnessValue,
                                                   num_store=number_of_parents_mating)
     #Generated offspring
    children = gc.crossover_uniform(parents=parents, childrenSize=(populationSize[0]-parents.shape[0],number_of_parameters))
    
    #Increase mutations to create genetic diversity
    children_mutated = gc.mutation(children, number_of_parameters)
    
     '''
       Create a new population that will contain the parent previously selected based on the fitness value, 
       and the resulting offspring

     '''
    population[0:parents.shape[0], :] = parents 
    population[parents.shape[0]:,  :] = children_mutated
    populationHistory[(generation+1)*number_of_parents:(generation+1)*number_of_parents+number_of_parents,:]=population
    

#The best solution for the final iteration 
fitness = gc.fitness_compute(population=population,dMatrixTrain=dtrain, dMatrixtest=dtest, y_test=te_y)

bestFitnessIndex = np.where(fitness == np.min(fitness))[0][0]
print("Best fitness is =", fitness[bestFitnessIndex])

print("Best parameters are:")
print('depth=',        int(population[bestFitnessIndex][0]))
print('iterations=',         population[bestFitnessIndex][1])
print('learning_rate=',            (population[bestFitnessIndex][2])) 
print('l2_leaf_reg=',     population[bestFitnessIndex][3])
print('bagging_temperature=',                population[bestFitnessIndex][4])
print('random_strength=',            population[bestFitnessIndex][5])

D:\APP.projection\anac_d\anaconda3\lib\site-packages\catboost\core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


This is number 0 generation
This is number 1 generation
This is number 2 generation
This is number 3 generation
This is number 4 generation
Best fitness is = 0.066133
Best parameters are:
depth= 9
iterations= 333.0
learning_rate= 0.52
l2_leaf_reg= 5.0
bagging_temperature= 9.43
random_strength= 9.43


In [5]:
a=[]
b=[]
print("\nthis-paper-model")

print("\nGA-xgboost Regression:")
xgb_model=xgb.XGBRegressor(n_estimators=600,
learning_rate= 0.487,
max_depth= 11,
min_child_weight= 1.803,
gamma= 5.684,
subsample= 0.713,
colsample_bytree= 0.379,)
xgb_model.fit(tr_x1,tr_y1)

y_pred1 = xgb_model.predict(tr_x1)
y_pred1_1 = xgb_model.predict(te_x1)



print("\nGA-catboost Regression:")
cb = catboost.CatBoostRegressor(silent=True,
             depth= 9,
iterations= 333.0,
learning_rate= 0.52,
l2_leaf_reg= 5.0,
bagging_temperature= 9.43,
random_strength= 9.43,)
cb.fit(tr_x,tr_y)
y_pred2 = cb.predict(tr_x)
y_pred2_1 = cb.predict(te_x)
u1=[]
pp=0
while pp <1:
    yy1=y_pred1-y_pred2
    y_pred3=pp*yy1+y_pred2
    u=mape(y_pred3,tr_y)
    u1.append(u)
    pp+=0.01
u2=pd.DataFrame(u1)
mm=1
for i in range(0,100):
    if u2.iloc[i,0]<mm:
        mm=u2.iloc[i,0]
        p=i
        i+=1
    else:
        i+=1
p
aa=p*0.01
yy2=y_pred1_1-y_pred2_1
y_pred3=aa*yy2+y_pred2_1
u=mape(y_pred3,te_y)
a.append(u)


this-paper-model

GA-xgboost回归:

GA-catboost回归:


D:\APP.projection\anac_d\anaconda3\lib\site-packages\catboost\core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [6]:

print("\nGA-xgboost Regression:")
xgb_model=xgb.XGBRegressor(n_estimators=600,
learning_rate= 0.487,
max_depth= 11,
min_child_weight= 1.803,
gamma= 5.684,
subsample= 0.713,
colsample_bytree= 0.379,)
xgb_model.fit(tr_x1,tr_y1)

y_pred1 = xgb_model.predict(tr_x1)
y_pred1_1 = xgb_model.predict(te_x1)

print("GA test set average absolute percentage error :{:.3f}".format(mape(y_pred1_1, te_y)))
print("GA test set absolute error of ga:{:.3f}".format(mean_absolute_error(te_y,y_pred1_1)))

XG_Mape=mape(y_pred1_1,te_y1)
XG_Mae=mean_absolute_error(te_y1,y_pred1_1)
a.append(XG_Mape)
b.append(XG_Mae)

print("\nGA-catboost Regression:")
cb = catboost.CatBoostRegressor(silent=True,
             depth= 9,
iterations= 333.0,
learning_rate= 0.52,
l2_leaf_reg= 5.0,
bagging_temperature= 9.43,
random_strength= 9.43,)
cb.fit(tr_x,tr_y)
y_pred2 = cb.predict(tr_x)
y_pred2_1 = cb.predict(te_x)

print("CB test set average absolute percentage error :{:.3f}".format(mape(y_pred2_1, te_y)))
print("CB test set absolute error:{:.3f}".format(mean_absolute_error(te_y,y_pred2_1)))

cb_Mape=mape(y_pred2_1,te_y)
cb_Mae=mean_absolute_error(te_y,y_pred2_1)
a.append(cb_Mape)
b.append(cb_Mae)




GA-xgboost回归:
ga测试集平均绝对百分比误差:0.064
ga测试集绝对误差:26178.796

GA-catboost回归:


D:\APP.projection\anac_d\anaconda3\lib\site-packages\catboost\core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


cb测试集平均绝对百分比误差:0.063
cb测试集绝对误差:26166.174


In [7]:
print("\nxgboost Regression:")
xgb_model=xgb.XGBRegressor()
xgb_model.fit(tr_x1,tr_y1)

y_pred1 = xgb_model.predict(tr_x1)
y_pred1_1 = xgb_model.predict(te_x1)

print("GA test set average absolute percentage error:{:.3f}".format(mape(y_pred1_1, te_y)))
print("GA test set absolute error of ga:{:.3f}".format(mean_absolute_error(te_y,y_pred1_1)))

XG_Mape1=mape(y_pred1_1,te_y1)
XG_Mae1=mean_absolute_error(te_y1,y_pred1_1)
a.append(XG_Mape1)
b.append(XG_Mae1)


xgboost回归:
ga测试集平均绝对百分比误差:0.066
ga测试集绝对误差:27541.667


In [8]:
print("\ncatboost Regression:")
cb = catboost.CatBoostRegressor(silent=True)
cb.fit(tr_x,tr_y)
y_pred2 = cb.predict(tr_x)
y_pred2_1 = cb.predict(te_x)

print("CB test set average absolute percentage error:{:.3f}".format(mape(y_pred2_1, te_y)))
print("CB test set absolute error:{:.3f}".format(mean_absolute_error(te_y,y_pred2_1)))

cb_Mape1=mape(y_pred2_1,te_y)
cb_Mae1=mean_absolute_error(te_y,y_pred2_1)
a.append(cb_Mape1)
b.append(cb_Mae1)


catboost回归:


D:\APP.projection\anac_d\anaconda3\lib\site-packages\catboost\core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


cb测试集平均绝对百分比误差:0.075
cb测试集绝对误差:31392.204


In [74]:
a

[0.06284377996992803,
 0.06401504789793909,
 0.0631751617839623,
 0.06638135476975367,
 0.07462374004656085]

In [75]:
# Dataset stitching
model_name=['paper_model','select_GA_xgboost','select_GA_catboost','xgboost','catboost']
model_name=pd.DataFrame(model_name)
model_name.rename(columns={0:'model_name'},inplace=True)
mape=pd.DataFrame(a)
mape.rename(columns={0:'MAPE'},inplace=True)
union_model=pd.concat([model_name,mape],axis=1)

In [76]:
union_model#Advantages of this paper model

,model_name,MAPE
0,paper_model,0.062844
1,select_GA_xgboost,0.064015
2,select_GA_catboost,0.063175
3,xgboost,0.066381
4,catboost,0.074624


In [77]:
union_model.to_excel('D:\\pyplt\\union_model.xlsx')#Export the dataset